In [9]:
import pandas as pd
import mysql.connector

def create_database():
    # Connect to the default database
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="1234"
    )
    conn.autocommit = True
    curr = conn.cursor()

    # Drop and create the projectp0 database
    curr.execute("DROP DATABASE IF EXISTS projectp0")
    curr.execute("CREATE DATABASE projectp0")

    # Close connection to the default database
    curr.close()
    conn.close()

    # Connect to the projectp0 database
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="1234",
        database="projectp0"
    )
    curr = conn.cursor()

    return curr, conn


In [11]:
curr, conn = create_database()


In [12]:
check_database_exists("projectp0")


Database 'projectp0' exists.


In [13]:
def drop_tables(curr, conn):
    try:
        for query in drop_table_queries:
            curr.execute(query)
            conn.commit()
    except Exception as e:
        print(f"Error dropping tables: {e}")

def create_tables(curr, conn):
    try:
        for query in create_table_queries:
            curr.execute(query)
            conn.commit()
    except Exception as e:
        print(f"Error creating tables: {e}")


In [14]:
# Load data
customer_data = pd.read_json("customers.json")
transaction_data = pd.read_json("transaction_logs.json")


In [15]:
# Create database and tables
curr, conn = create_database()

customer_table_create = """
CREATE TABLE IF NOT EXISTS Customer (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    country VARCHAR(100),
    city VARCHAR(100)
);
"""
curr.execute(customer_table_create)
conn.commit()

# Modify Product table to include ecommerce_website_name
product_table_create = """
CREATE TABLE IF NOT EXISTS Product (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    product_category VARCHAR(100),
    price DECIMAL(10, 2),
    ecommerce_website_name VARCHAR(255)
);
"""
curr.execute(product_table_create)
conn.commit()

order_table_create = """
CREATE TABLE IF NOT EXISTS Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    qty INT,
    datetime TEXT,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id),
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
);
"""
curr.execute(order_table_create)
conn.commit()

payment_table_create = """
CREATE TABLE IF NOT EXISTS Payment (
    payment_txn_id VARCHAR(255) PRIMARY KEY,
    order_id INT,
    payment_type VARCHAR(50),
    payment_txn_success CHAR(1),
    failure_reason VARCHAR(255),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);
"""
curr.execute(payment_table_create)
conn.commit()


In [16]:
customer_data.head()

,customer_id,customer_name,country,city
0,9780,Jessica Hart,Martinique,South William
1,9169,Amy Clark,Kazakhstan,Davisville
2,6631,Danielle Elliott,Armenia,East Danny
3,8772,Robin Mendoza,Uganda,Johnsonbury
4,8913,Abigail Miller,Namibia,Wrightchester


In [18]:
transaction_data.head()

,order_id,customer_id,product_id,product_name,product_category,payment_type,qty,price,datetime,ecommerce_website_name,payment_txn_id,payment_txn_success,failure_reason
0,1,1731,8609,Central home & kitche,Home & Kitchen,Wallet,25,64216,2021-07-26 20:16:14,www.bestbuy.com,a395e8c7,N,Card Expired
1,2,5539,2409,Upon sport,Sports,Card,34,69877,2021-08-25 03:41:32,www.flipkart.com,b40791f8,N,Invalid CVV
2,3,3507,1145,Across stationer,Stationery,Card,28,35491,2024-06-21 21:39:21,www.target.com,7fb6cca9,N,Blocked Card
3,4,4826,8375,Start home & kitche,Home & Kitchen,Card,33,39540,2022-09-17 13:52:11,www.amazon.in,cae677f3,N,Transaction Timeout
4,5,6538,2149,Draw beaut,Beauty,Card,27,80635,2022-03-09 07:58:08,www.etsy.com,fbce1649,N,Invalid CVV


In [19]:
# Insert data into Customer table
customer_insert_query = """
INSERT INTO Customer (customer_id, customer_name, country, city)
VALUES (%s, %s, %s, %s);
"""
try:
    for i, row in customer_data.iterrows():
        curr.execute(customer_insert_query, list(row))
    conn.commit()
    print("Data inserted into Customer table successfully.")
except Exception as e:
    print(f"Error inserting into Customer table: {e}")


Data inserted into Customer table successfully.


In [20]:
# Insert data into Product table
product_insert_query = """
INSERT INTO Product (product_id, product_name, product_category, price, ecommerce_website_name)
VALUES (%s, %s, %s, %s, %s);
"""
try:
    product_table = transaction_data[["product_id", "product_name", "product_category", "price", "ecommerce_website_name"]].drop_duplicates(subset=['product_id'])
    for i, row in product_table.iterrows():
        curr.execute(product_insert_query, list(row))
    conn.commit()
    print("Data inserted into Product table successfully.")
except Exception as e:
    print(f"Error inserting into Product table: {e}")


Data inserted into Product table successfully.


In [21]:
# Insert data into Orders table
order_insert_query = """
INSERT INTO Orders (order_id, customer_id, product_id, qty, datetime)
VALUES (%s, %s, %s, %s, %s);
"""
try:
    orders_table = transaction_data[["order_id", "customer_id", "product_id", "qty", "datetime"]]
    for i, row in orders_table.iterrows():
        curr.execute(order_insert_query, list(row))
    conn.commit()
    print("Data inserted into Orders table successfully.")
except Exception as e:
    print(f"Error inserting into Orders table: {e}")


Data inserted into Orders table successfully.


In [22]:
# Insert data into Payment table
payment_insert_query = """
INSERT INTO Payment (payment_txn_id, order_id, payment_type, payment_txn_success, failure_reason)
VALUES (%s, %s, %s, %s, %s);
"""
try:
    payment_table = transaction_data[["payment_txn_id", "order_id", "payment_type", "payment_txn_success", "failure_reason"]]
    for i, row in payment_table.iterrows():
        curr.execute(payment_insert_query, list(row))
    conn.commit()
    print("Data inserted into Payment table successfully.")
except Exception as e:
    print(f"Error inserting into Payment table: {e}")


Data inserted into Payment table successfully.


In [23]:
# Close connection
curr.close()
conn.close()
print("Database connection closed.")


Database connection closed.


In [24]:
import mysql.connector

def fetch_sample_data(table_name, connection):
    try:
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(query)
        result = cursor.fetchall()
        print(f"\nSample data from {table_name} table:")
        for row in result:
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def main():
    # Connect to the database
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="1234",
        database="projectp0"
    )

    # Fetch and display sample data from each table
    tables = ["Customer", "Product", "Orders", "Payment"]
    for table in tables:
        fetch_sample_data(table, conn)

    # Close connection
    conn.close()

if __name__ == "__main__":
    main()



Sample data from Customer table:
(102, 'Kenneth Singh', 'Sweden', 'Davidhaven')
(106, 'Mr. James Mendez', 'British Indian Ocean Territory (Chagos Archipelago)', 'Paulmouth')
(109, 'Joseph Martin', 'Somalia', 'Port Vickiemouth')
(135, 'Joann Romero', 'Timor-Leste', 'Port Raymondmouth')
(144, 'Robert Reeves', 'Wallis and Futuna', 'Lake Jason')

Sample data from Product table:
(201, 'Song toy', 'Toys', Decimal('36403.00'), 'www.target.com')
(202, 'Enjoy stationer', 'Stationery', Decimal('26248.00'), 'www.bestbuy.com')
(203, 'Represent toy', 'Toys', Decimal('47723.00'), 'www.bestbuy.com')
(206, 'Law electronic', 'Electronics', Decimal('4177.00'), 'www.flipkart.com')
(208, 'Issue electronic', 'Electronics', Decimal('25652.00'), 'www.target.com')

Sample data from Orders table:
(1, 1731, 8609, 25, '2021-07-26 20:16:14')
(2, 5539, 2409, 34, '2021-08-25 03:41:32')
(3, 3507, 1145, 28, '2024-06-21 21:39:21')
(4, 4826, 8375, 33, '2022-09-17 13:52:11')
(5, 6538, 2149, 27, '2022-03-09 07:58:08')



In [25]:
# Close connection
curr.close()
conn.close()
print("Database connection closed.")


Database connection closed.
